In [34]:
import tensorflow as tf
import matplotlib.pyplot as plt
train_dir = './Planets and Moons/train'
val_dir = './Planets and Moons/test'

In [35]:
# On prend celui qui 
planetes = dict()
for planete in tf.io.gfile.listdir(train_dir):
    planetes[planete] = len(tf.io.gfile.listdir(train_dir + '/' + planete))
print(planetes)   
print(min(planetes.values()))

{'Earth': 74, 'Jupiter': 74, 'MakeMake': 74, 'Mars': 74, 'Mercury': 74, 'Moon': 73, 'Neptune': 74, 'Pluto': 74, 'Saturn': 74, 'Uranus': 74, 'Venus': 74}
73


In [36]:
min_images = min(planetes.values())

for planete in tf.io.gfile.listdir(train_dir):
    images = tf.io.gfile.listdir(train_dir + '/' + planete)
    for image in images[min_images:]:
        tf.io.gfile.remove(train_dir + '/' + planete + '/' + image)
        print(len(tf.io.gfile.listdir(train_dir + '/' + planete)))

73
73
73
73
73
73
73
73
73
73


In [21]:
if len(tf.io.gfile.listdir(val_dir)) == 0:
    tf.io.gfile.rmtree(val_dir)
    tf.io.gfile.makedirs(val_dir)
    
    for planete in tf.io.gfile.listdir(train_dir):
        i = len(tf.io.gfile.listdir(train_dir + '/' + planete))
        tf.io.gfile.makedirs(val_dir + '/' + planete)
        images = tf.io.gfile.listdir(train_dir + '/' + planete)
        y = 0
        for image in images:
            if y <= i/2:
                tf.io.gfile.copy(train_dir + '/' + planete + '/' + image, val_dir + '/' + planete + '/' + image)
                tf.io.gfile.remove(train_dir + '/' + planete + '/' + image)
                y+= 1

In [3]:
len(tf.io.gfile.listdir(train_dir))

print("Sur le dossier train :")

for fruit in tf.io.gfile.listdir(train_dir):
    print(f"{fruit} - {len(tf.io.gfile.listdir(train_dir + '/' + fruit))} images")
    
print("\nSur le dossier test :")

for fruit in tf.io.gfile.listdir(val_dir):
    print(f"{fruit} - {len(tf.io.gfile.listdir(val_dir + '/' + fruit))} images")

Sur le dossier train :
Earth - 74 images
Jupiter - 74 images
MakeMake - 74 images
Mars - 74 images
Mercury - 74 images
Moon - 73 images
Neptune - 74 images
Pluto - 74 images
Saturn - 74 images
Uranus - 74 images
Venus - 74 images

Sur le dossier test :
Earth - 75 images
Jupiter - 75 images
MakeMake - 75 images
Mars - 75 images
Mercury - 75 images
Moon - 75 images
Neptune - 75 images
Pluto - 75 images
Saturn - 75 images
Uranus - 75 images
Venus - 75 images


In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    seed = 42
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical',
    seed = 42
)

Found 813 images belonging to 11 classes.
Found 825 images belonging to 11 classes.


In [5]:
inputs = tf.keras.layers.Input(shape=(150, 150, 3))

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D(2)(x)

x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2)(x)

x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(2)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496 

In [6]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer='adam')

history = model.fit(train_gen, epochs=25, validation_data=val_gen)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\Enzo Daval\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\Enzo Daval\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\Enzo Daval\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\Enzo Daval\AppData\L

In [ ]:
def diagnostic_learning_curves():
  # Retrieve a list of accuracy results on training and validation data
  # sets for each training epoch
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  # Retrieve a list of list results on training and validation data
  # sets for each training epoch
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  # Get number of epochs
  epochs = range(len(acc))

  # Plot training and validation accuracy per epoch
  plt.plot(epochs, acc)
  plt.plot(epochs, val_acc)
  plt.title('Training and validation accuracy')

  plt.figure()

  # Plot training and validation loss per epoch
  plt.plot(epochs, loss)
  plt.plot(epochs, val_loss)
  plt.title('Training and validation loss')

In [ ]:
diagnostic_learning_curves()